X

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import glob
import json
import warnings
import datetime as dt
warnings.filterwarnings('ignore')


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [2]:
os.getcwd()
#os.chdir()

'/Users/ivytong/H1B Dashboard'

In [3]:
use_columns_20212020 = ['CASE_NUMBER', 'CASE_STATUS','DECISION_DATE', 'VISA_CLASS', 'JOB_TITLE', 'SOC_CODE',
       'SOC_TITLE','EMPLOYER_NAME','NAICS_CODE','LAWFIRM_NAME_BUSINESS_NAME','WORKSITE_CITY',
       'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 'WAGE_RATE_OF_PAY_FROM',
       'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE',
       'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_OES_YEAR','PW_OTHER_SOURCE']

use_columns_2019 = ['CASE_NUMBER', 'CASE_STATUS','DECISION_DATE', 'VISA_CLASS', 'JOB_TITLE', 'SOC_CODE',
       'SOC_TITLE','EMPLOYER_NAME','NAICS_CODE', 'AGENT_ATTORNEY_LAW_FIRM_BUSINESS_NAME', 'WORKSITE_CITY_1',
        'WORKSITE_STATE_1','WORKSITE_POSTAL_CODE_1','WAGE_RATE_OF_PAY_FROM_1', 
        'WAGE_RATE_OF_PAY_TO_1', 'WAGE_UNIT_OF_PAY_1', 'PREVAILING_WAGE_1', 
        'PW_UNIT_OF_PAY_1', 'PW_WAGE_LEVEL_1', 'PW_OES_YEAR_1', 'PW_OTHER_SOURCE_1']

use_columns_20182017 = ['CASE_NUMBER', 'CASE_STATUS','DECISION_DATE', 'VISA_CLASS', 'JOB_TITLE', 'SOC_CODE',
       'SOC_NAME','EMPLOYER_NAME','NAICS_CODE','AGENT_ATTORNEY_NAME','WORKSITE_CITY', 
       'WORKSITE_STATE', 'WORKSITE_POSTAL_CODE', 'WAGE_RATE_OF_PAY_FROM',
       'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY', 'PREVAILING_WAGE',
       'PW_UNIT_OF_PAY', 'PW_WAGE_LEVEL', 'PW_SOURCE_YEAR','PW_SOURCE']

# df.columns[np.r_[4,35,36,85,96,97,98]]

In [4]:
# df = pd.read_csv('/Users/ivytong/H1B Dashboard/Data/Final/FY2021.csv', usecols = use_columns_20212020)
# df.info()

In [5]:
'''
Define a number of functions to clean the data
'''

def clean_zip_code(zipcode):
    return re.sub('[^0-9]','', zipcode)[:5]


def clean_code(code):
    return code.split('.')[0][:7]

def clean_title(title):
    step1 = re.sub('[^A-Za-z\s]','', title.strip())
    step2 = re.sub('[Ss]$','', step1.strip())
    final_title = re.sub('\s+',' ', step2.strip())
    return final_title.upper().title().strip()

STATE_ABBREV_LOOKUP = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

def get_state_abbrev(state):
    if len(state) == 2:
        return state.upper()
    elif state.lower().title().replace('The','').strip() in STATE_ABBREV_LOOKUP:
        return STATE_ABBREV_LOOKUP[state.lower().title()]   
    else:
        return 'No Info'
    
RE_CITY = re.compile(r"^([a-zA-Z\']+(\.|\,)(\s?|\-))?(([a-zA-Z\']+(\s+|\-))*)([a-zA-Z\']+)$")
def clean_city(city):
    step1 = city.replace(',','')
    step2 = step1.replace('&nbsp','')
    step3 = step2.replace('D.C.','')
    result = RE_CITY.search(step3.strip())
    if result:
        return result[0].title()
    return 'No Info'


# wage_unit_conversion = {
#     'Year': 1,
#     "Month" : 12, 
#     "Hour" : 2080,
#     "Bi-Weekly" : 26,
#     "Week" : 52,
#     np.nan:1
# }
# def get_wage_unit(wage_unit):
#     return wage_unit_conversion[wage_unit]

hour_unit_conversion = {
    'Year': 2080,
    "Month" : 175, #2080/12
    "Hour" : 1,
    "Bi-Weekly" : 80,
    "Week" : 40,
    np.nan : 2080
}

def get_wage_unit(wage_unit):
    return hour_unit_conversion[wage_unit]

def get_hourly_wage(wage_per_hour):
    if wage_per_hour >=7.25 and wage_per_hour <= 3500:
        return wage_per_hour    
    return 7.25


In [6]:
def company_name_replace(company_name,n):
    def internal(string):
        string = company_name.lower().strip()
        string = re.sub(r'(\,\s*|\s+)p(\.)?l(\.)?l(\.)?c(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)p(\.)?l(\.)?c(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)l(\.)?l(\.)?c(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)l(\.)?p(\.)?a(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)a(\.)?p(\.)?c(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)l(\.)?l(\.)?p(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)l(\.)?l(\.)?o(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)l(\.)?c(\.)?c(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)u(\.)?s(\.)?a(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)inc(\.)?($|\W)', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)corp(\.)?($|\W)', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)assc(\.)?($|\W)', ' associates', string).strip()
        string = re.sub(r'(\,\s*|\s+)l(\.)?t(\.)?d(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)l(\.)?p(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)p(\.)?a(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)p(\.)?c(\.)?$', '', string).strip()
        string = re.sub(r'(\,\s*|\s+)o(\.)?d(\.)?$', '', string).strip()
        string = re.sub(r'(\(:.;(/)+-=*(\)`|$))', '', string).strip()
        string = re.sub(r'\.$', '', string).strip()
        string = re.sub(r'\-', ' ', string).strip()
        string = re.sub(r'[\?]*[\|]*[\/]*', '', string).strip()
        string = re.sub(r'\\*', '', string).strip()
        string = re.sub(r'group$', '', string).strip()
        string = re.sub(r'corporation$', '', string).strip()
        string = re.sub(r'incorporated$', '', string).strip()
        string = re.sub(r'office[s]*$', '', string).strip()
        string = re.sub(r'firm[s]*$', '', string).strip()
        string = re.sub(r'group$', '', string).strip()
        string = re.sub(r'americas$', '', string).strip()
        string = re.sub(r'\s+us$', '', string).strip()
        string = re.sub(r'\&', 'and', string).strip()
        string = re.sub(r'\,\s+and\s+', ' and ', string).strip()
        string = re.sub(r'\s+([A-Za-z])\.?\s+', r' \1. ', string)
        string = re.sub(r'\s+', ' ', string).strip()
        string = re.sub(r'^[,]+$', 'No Info', string).strip()

        return string.title().strip()
    
    result = company_name
    for i in range(n):
        result = internal(result)
    
    return result



In [7]:
def clean_data_20212020(data):
    data['DECISION_DATE'] = data['DECISION_DATE'].dt.date
    data['CASE_STATUS'] = data['CASE_STATUS'].str.lower().str.title()
    data = data[(data['CASE_STATUS'] == "Certified") & ((data['VISA_CLASS']== 'H-1B')| (data['VISA_CLASS']== 'H-1B1 Chile')| (data['VISA_CLASS']== 'H-1B1 Singapore'))].drop_duplicates(subset=['CASE_NUMBER'])
    
    for x in ['EMPLOYER_NAME','LAWFIRM_NAME_BUSINESS_NAME','WORKSITE_CITY','WORKSITE_STATE','WORKSITE_POSTAL_CODE','JOB_TITLE','PW_WAGE_LEVEL',
             'PW_OES_YEAR','PW_OTHER_SOURCE']:
        data[x] = data[x].fillna('No Info')
        
    
    data['SOC_CODE'] = data['SOC_CODE'].apply(lambda x : clean_code(x))
    data['SOC_TITLE'] = data['SOC_TITLE'].apply(lambda x : clean_title(x))
    data['NAICS_CODE'] = data['NAICS_CODE'].apply(lambda x : clean_code(str(x)))   
    data['WORKSITE_STATE'] = data['WORKSITE_STATE'].apply(lambda x : get_state_abbrev(x))
    data['WORKSITE_CITY'] = data['WORKSITE_CITY'].apply(lambda x : clean_city(x))
    data['WORKSITE_POSTAL_CODE'] = data['WORKSITE_POSTAL_CODE'].apply(lambda x : clean_zip_code(x))   
    data['WORKSITE_POSTAL_CODE'] = data['WORKSITE_POSTAL_CODE'].str.zfill(5)
    
    data['WAGE_UNIT'] =  data['WAGE_UNIT_OF_PAY'].apply(lambda x : get_wage_unit(x))
    data['WAGE1_HOURLY_RATE'] = data['WAGE_RATE_OF_PAY_FROM'] / data['WAGE_UNIT']
    data['WAGE2_HOURLY_RATE'] = data['WAGE_RATE_OF_PAY_TO'] / data['WAGE_UNIT']    
    data['WAGE1'] = data['WAGE1_HOURLY_RATE'].apply(lambda x: get_hourly_wage(x) * 2080) 
    data['WAGE2'] = data['WAGE1_HOURLY_RATE'].apply(lambda x: get_hourly_wage(x) * 2080) 
    
    data['ANNUAL_WAGE'] = data['WAGE2'] if data['WAGE1'].empty else data['WAGE1'] 
    data['PW_UNIT'] = data['PW_UNIT_OF_PAY'].apply(lambda x : get_wage_unit(x))
    data['ANNUAL_PW_RATE'] = data['PREVAILING_WAGE'] / data['PW_UNIT']
    data['ANNUAL_PW'] = data['ANNUAL_PW_RATE'].apply(lambda x : get_hourly_wage(x) * 2080)
    
    
    data['JOB_TITLE'] = data['JOB_TITLE'].apply(lambda x : clean_title(x))
    data['EMPLOYER'] = data['EMPLOYER_NAME'].apply(lambda x : company_name_replace(x,2))
    data['EMPLOYER_2'] = data['EMPLOYER'] .apply(lambda x: 'Amazon' if 'Amazon' in x else x)
#     data.loc[data['EMPLOYER'].str.contains('AMAZON', case= False), 'EMPLOYER'] = 'AMAZON'
    data['AGENT_LAWFIRM'] = data['LAWFIRM_NAME_BUSINESS_NAME'].apply(lambda x : company_name_replace(x,2))
    
    stem_list = set(pd.read_excel('stem.xlsx')['OCC_CODE'])
    data['STEM'] = data['SOC_CODE'].apply(lambda x : 1 if x in stem_list else 0)
    data['SOC_CODE'] = data['SOC_CODE'].apply(lambda x : "SOC " + clean_code(x))
    
    data = data.drop(columns = ['CASE_NUMBER','WAGE_UNIT','WAGE1',
                                'WAGE2','PW_UNIT_OF_PAY','PW_UNIT','PREVAILING_WAGE',
                               'WAGE_RATE_OF_PAY_FROM','WAGE_RATE_OF_PAY_TO','WAGE_UNIT_OF_PAY','EMPLOYER_NAME','LAWFIRM_NAME_BUSINESS_NAME',
                               'WAGE1_HOURLY_RATE','WAGE2_HOURLY_RATE','ANNUAL_PW_RATE'])

    
    return data

for file in glob.glob('Data/Final/FY202*.csv'):
    filename = file.split('/')[-1]
    df = pd.read_csv(file, usecols = use_columns_20212020,parse_dates=['DECISION_DATE'])
    clean_data_20212020(df).to_csv(os.path.join(os.getcwd(),'Download/',filename), index = False)
    


In [8]:
def clean_data_2019(data):
    data['DECISION_DATE'] = data['DECISION_DATE'].dt.date
    data['CASE_STATUS'] = data['CASE_STATUS'].str.lower().str.title()
    data = data[(data['CASE_STATUS'] == "Certified") & ((data['VISA_CLASS']== 'H-1B')| (data['VISA_CLASS']== 'H-1B1 Chile')| (data['VISA_CLASS']== 'H-1B1 Singapore'))].drop_duplicates(subset=['CASE_NUMBER'])
    
    for x in ['EMPLOYER_NAME','AGENT_ATTORNEY_LAW_FIRM_BUSINESS_NAME','WORKSITE_CITY_1','WORKSITE_STATE_1','WORKSITE_POSTAL_CODE_1','JOB_TITLE']:
        data[x] = data[x].fillna('No Info')
    
    data['PW_WAGE_LEVEL'] = data['PW_WAGE_LEVEL_1'].fillna('No Info')
    data['PW_OES_YEAR'] = data['PW_OES_YEAR_1'].fillna('No Info')
    data['PW_OTHER_SOURCE'] = data['PW_OTHER_SOURCE_1'].fillna('No Info')
    
    data['SOC_CODE'] = data['SOC_CODE'].apply(lambda x : clean_code(x))
    data['SOC_TITLE'] = data['SOC_TITLE'].apply(lambda x : clean_title(x))
    data['NAICS_CODE'] = data['NAICS_CODE'].apply(lambda x : clean_code(str(x)) ) 
    data['WORKSITE_STATE'] = data['WORKSITE_STATE_1'].apply(lambda x : get_state_abbrev(x))
    data['WORKSITE_CITY'] = data['WORKSITE_CITY_1'].apply(lambda x : clean_city(x))
    data['WORKSITE_POSTAL_CODE'] = data['WORKSITE_POSTAL_CODE_1'].apply(lambda x : clean_zip_code(x))   
    data['WORKSITE_POSTAL_CODE'] = data['WORKSITE_POSTAL_CODE'].str.zfill(5)
    
    
    data['WAGE_UNIT'] =  data['WAGE_UNIT_OF_PAY_1'].apply(lambda x : get_wage_unit(x))
    data['WAGE1_HOURLY_RATE'] = data['WAGE_RATE_OF_PAY_FROM_1'] / data['WAGE_UNIT']
    data['WAGE2_HOURLY_RATE'] = data['WAGE_RATE_OF_PAY_TO_1'] / data['WAGE_UNIT']
    data['WAGE1'] = data['WAGE1_HOURLY_RATE'].apply(lambda x: get_hourly_wage(x)*2080) 
    data['WAGE2'] = data['WAGE1_HOURLY_RATE'].apply(lambda x: get_hourly_wage(x)*2080) 
    data['ANNUAL_WAGE'] = data['WAGE2'] if data['WAGE1'].empty else data['WAGE1'] 
    data['PW_UNIT'] = data['PW_UNIT_OF_PAY_1'].apply(lambda x : get_wage_unit(x))
    data['ANNUAL_PW_RATE'] = data['PREVAILING_WAGE_1'] / data['PW_UNIT']
    data['ANNUAL_PW'] = data['ANNUAL_PW_RATE'].apply(lambda x : get_hourly_wage(x)*2080)
 
 
    
    data['JOB_TITLE'] = data['JOB_TITLE'].apply(lambda x : clean_title(x))
    data['EMPLOYER'] = data['EMPLOYER_NAME'].apply(lambda x : company_name_replace(x,2))
    data['EMPLOYER_2'] = data['EMPLOYER'] .apply(lambda x: 'Amazon' if 'Amazon' in x else x)
    data['AGENT_LAWFIRM'] = data['AGENT_ATTORNEY_LAW_FIRM_BUSINESS_NAME'].apply(lambda x : company_name_replace(x,2))
    
    stem_list = set(pd.read_excel('stem.xlsx')['OCC_CODE'])
    data['STEM'] = data['SOC_CODE'].apply(lambda x : 1 if x in stem_list else 0)
    data['SOC_CODE'] = data['SOC_CODE'].apply(lambda x : "SOC " + clean_code(x))
    
    data = data.drop(columns = ['CASE_NUMBER','WAGE_UNIT','WAGE1','WORKSITE_CITY_1','WORKSITE_STATE_1','WORKSITE_POSTAL_CODE_1',
                                'WAGE2','PW_UNIT_OF_PAY_1','PW_UNIT','PREVAILING_WAGE_1',
                               'WAGE_RATE_OF_PAY_FROM_1','WAGE_RATE_OF_PAY_TO_1','WAGE_UNIT_OF_PAY_1','EMPLOYER_NAME','AGENT_ATTORNEY_LAW_FIRM_BUSINESS_NAME','PW_WAGE_LEVEL_1','PW_OES_YEAR_1','PW_OTHER_SOURCE_1',
                               'WAGE1_HOURLY_RATE','WAGE2_HOURLY_RATE','ANNUAL_PW_RATE'])

    
    return data

file = 'Data/Final/FY2019.csv'
filename = file.split('/')[-1]
df = pd.read_csv(file, usecols = use_columns_2019, parse_dates=['DECISION_DATE'])
clean_data_2019(df).to_csv(os.path.join(os.getcwd(),'Download/',filename), index = False)
    

In [9]:
def clean_data_20182017(data):
 
    data['DECISION_DATE'] = data['DECISION_DATE'].dt.date
    data['CASE_STATUS'] = data['CASE_STATUS'].str.lower().str.title()
    data = data[(data['CASE_STATUS'] == "Certified") & ((data['VISA_CLASS']== 'H-1B')| (data['VISA_CLASS']== 'H-1B1 Chile')| (data['VISA_CLASS']== 'H-1B1 Singapore'))].drop_duplicates(subset=['CASE_NUMBER'])
    
    for x in ['EMPLOYER_NAME','AGENT_ATTORNEY_NAME','WORKSITE_CITY','WORKSITE_STATE','WORKSITE_POSTAL_CODE','JOB_TITLE','PW_WAGE_LEVEL',
             'PW_SOURCE_YEAR','PW_SOURCE']:
        data[x] = data[x].fillna('No Info')


    data['SOC_CODE'] = data['SOC_CODE'].apply(lambda x : clean_code(x))
    data['SOC_TITLE'] = data['SOC_NAME'].apply(lambda x : clean_title(x))
    data['NAICS_CODE'] = data['NAICS_CODE'].apply(lambda x : clean_code(str(x)) )   
    data['WORKSITE_STATE'] = data['WORKSITE_STATE'].apply(lambda x : get_state_abbrev(x))
    data['WORKSITE_CITY'] = data['WORKSITE_CITY'].apply(lambda x : clean_city(x))
    data['WORKSITE_POSTAL_CODE'] = data['WORKSITE_POSTAL_CODE'].apply(lambda x : clean_zip_code(x))   
    data['WORKSITE_POSTAL_CODE'] = data['WORKSITE_POSTAL_CODE'].str.zfill(5)
    
    data['WAGE_UNIT'] =  data['WAGE_UNIT_OF_PAY'].apply(lambda x : get_wage_unit(x))
    data['WAGE1_HOURLY_RATE'] = data['WAGE_RATE_OF_PAY_FROM'] / data['WAGE_UNIT']
    data['WAGE2_HOURLY_RATE'] = data['WAGE_RATE_OF_PAY_TO'] / data['WAGE_UNIT']    
    data['WAGE1'] = data['WAGE1_HOURLY_RATE'].apply(lambda x: get_hourly_wage(x) * 2080)
    data['WAGE2'] = data['WAGE1_HOURLY_RATE'].apply(lambda x: get_hourly_wage(x) *2080)
    data['ANNUAL_WAGE'] = data['WAGE2'] if data['WAGE1'].empty else data['WAGE1'] 
    
    data['PW_UNIT'] = data['PW_UNIT_OF_PAY'].apply(lambda x : get_wage_unit(x))
    data['ANNUAL_PW_RATE'] = data['PREVAILING_WAGE'] / data['PW_UNIT']
    data['ANNUAL_PW'] = data['ANNUAL_PW_RATE'].apply(lambda x : get_hourly_wage(x) * 2080) 
    
    
    data['JOB_TITLE'] = data['JOB_TITLE'].apply(lambda x : clean_title(x))
    data['EMPLOYER'] = data['EMPLOYER_NAME'].apply(lambda x : company_name_replace(x,2))
    data['EMPLOYER_2'] = data['EMPLOYER'] .apply(lambda x: 'Amazon' if 'Amazon' in x else x)
    data['AGENT_LAWFIRM'] = data['AGENT_ATTORNEY_NAME'].apply(lambda x : company_name_replace(x,2))
    
    stem_list = set(pd.read_excel('stem.xlsx')['OCC_CODE'])
    data['STEM'] = data['SOC_CODE'].apply(lambda x : 1 if x in stem_list else 0)
    data['SOC_CODE'] = data['SOC_CODE'].apply(lambda x : "SOC " + clean_code(x))
    
    data = data.drop(columns = ['CASE_NUMBER','WAGE_UNIT','WAGE1','SOC_NAME',
                                'WAGE2','PW_UNIT_OF_PAY','PW_UNIT','PREVAILING_WAGE',
                               'WAGE_RATE_OF_PAY_FROM','WAGE_RATE_OF_PAY_TO','WAGE_UNIT_OF_PAY','EMPLOYER_NAME','AGENT_ATTORNEY_NAME',
                               'WAGE1_HOURLY_RATE','WAGE2_HOURLY_RATE','ANNUAL_PW_RATE'])


    
    return data



for file in glob.glob('Data/Final/FY201*.csv'):
    if file.split('/')[-1].split('.')[0][2:] in ['2017', '2018']:
        filename = file.split('/')[-1]
        df = pd.read_csv(file, usecols = use_columns_20182017,parse_dates=['DECISION_DATE'])
        clean_data_20182017(df).to_csv(os.path.join(os.getcwd(),'Download/',filename), index = False)
    